In [4]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
from PIL import Image, ImageTk, ImageOps, ImageDraw, ImageFont
import cv2
import numpy as np
import re
import os
import sys

class YSMAArtViewer:
    def __init__(self, root):
        self.root = root
        self.root.title("YSMA Art Collection Viewer")
        self.root.geometry("1000x800")

        # Configure for Jupyter compatibility
        self.is_jupyter = 'ipykernel' in sys.modules
        if self.is_jupyter:
            from IPython import get_ipython
            ipython = get_ipython()
            if ipython is not None:
                ipython.run_line_magic('matplotlib', 'inline')

        # User data
        self.current_user = None
        self.user_password = None
        self.user_category = None

        # Image data
        self.current_image = None
        self.original_image = None
        self.image_label = None
        self.image_path = None

        # Art categories and paths
        self.valid_categories = {
            "traditional": ["Benin", "Gelede-Headdress", "Ife-Royal-head-Bronze", "Male-Nok-Head", "Traditional-bell"],
            "modern": ["Honorable-man", "Masks", "The-Dancer", "The-dye-pit"],
            "contemporary": ["Ariya", "Common-Goal", "The-Hybrid-ideas", "The-way-home", "Unveiling-the-masquerade"]
        }
        
        self.category_paths = {
            "traditional": "C:\\Users\\maxos\\CSC418\\CA1\\Traditional-arts",
            "modern": "C:\\Users\\maxos\\CSC418\\CA1\\Modern-arts",
            "contemporary": "C:\\Users\\maxos\\CSC418\\CA1\\Contemporary-arts"
        }

        # Add background image
        self.set_background_image("C:\\Users\\maxos\\CSC418\\CA1\\Monkey.png")
        
        self.create_login_screen()

    def set_background_image(self, image_path):
        """Set a background image for the window"""
        try:
            bg_image = Image.open(image_path)
            bg_image = bg_image.resize((1000, 800), Image.LANCZOS)
            self.bg_photo = ImageTk.PhotoImage(bg_image)
            self.bg_label = tk.Label(self.root, image=self.bg_photo)
            self.bg_label.place(x=0, y=0, relwidth=1, relheight=1)
            self.bg_label.lower()
        except Exception as e:
            print(f"Could not load background image: {str(e)}")
            self.root.configure(bg="#f0f0f0")

    def create_login_screen(self):
        """Create the login/registration screen without a frame"""
        self.clear_window(keep_background=True)

        # Place widgets directly on the root window
        ttk.Label(self.root, text="YSMA Art Collection", font=("Aptos", 28)).place(relx=0.5, rely=0.3, anchor="center")
        
        # Username
        ttk.Label(self.root, text="Username:").place(relx=0.45, rely=0.4, anchor="e")
        self.username_entry = ttk.Entry(self.root)
        self.username_entry.place(relx=0.55, rely=0.4, anchor="w")

        # Password
        ttk.Label(self.root, text="Password:").place(relx=0.45, rely=0.45, anchor="e")
        self.password_entry = ttk.Entry(self.root, show="*")
        self.password_entry.place(relx=0.55, rely=0.45, anchor="w")

        # Category
        ttk.Label(self.root, text="Category:").place(relx=0.45, rely=0.5, anchor="e")
        self.category_var = tk.StringVar()
        self.category_combobox = ttk.Combobox(self.root, textvariable=self.category_var, 
                                              values=list(self.valid_categories.keys()))
        self.category_combobox.place(relx=0.55, rely=0.5, anchor="w")

        # Submit button
        ttk.Button(self.root, text="Enter", command=self.validate_user_input).place(relx=0.5, rely=0.6, anchor="center")

    def validate_user_input(self):
        """Validate user input before granting access"""
        username = self.username_entry.get().strip()
        password = self.password_entry.get().strip()
        category = self.category_var.get().lower()

        if not username or not password:
            messagebox.showerror("Invalid Input", "Username and password cannot be empty")
            return

        if category not in self.valid_categories:
            messagebox.showerror("Invalid Category", "Please select a valid category: traditional, modern, or contemporary")
            return

        self.current_user = username
        self.user_password = password
        self.user_category = category
        self.create_main_menu()

    def create_main_menu(self):
        """Create the main menu after successful login"""
        self.clear_window(keep_background=True)

        welcome_msg = f"Welcome {self.current_user}!\nPreferred Category: {self.user_category.title()}"
        ttk.Label(self.root, text=welcome_msg, font=("Aptos", 16)).pack(pady=10)

        category_frame = ttk.LabelFrame(self.root, text="Browse Collection", padding=10)
        category_frame.pack(fill="x", padx=10, pady=5)

        for subcategory in self.valid_categories[self.user_category]:
            ttk.Button(category_frame, text=subcategory, 
                      command=lambda sc=subcategory: self.load_category_image(sc)).pack(fill="x", pady=2)

        tools_frame = ttk.LabelFrame(self.root, text="Enhancement Tools", padding=10)
        tools_frame.pack(fill="x", padx=10, pady=5)

        ttk.Button(tools_frame, text="Basic Transformations", 
                  command=self.show_basic_transformations).pack(fill="x", pady=2)
        ttk.Button(tools_frame, text="Advanced Transformations", 
                  command=self.show_advanced_transformations).pack(fill="x", pady=2)

        ttk.Button(self.root, text="Logout", command=self.logout).pack(side="bottom", pady=10)

    def load_category_image(self, subcategory):
        """Load an image from the selected category"""
        try:
            img_path = os.path.join(self.category_paths[self.user_category], f"{subcategory}.jpg")
            if os.path.exists(img_path):
                img = Image.open(img_path)
            else:
                img = Image.new('RGB', (600, 400), color=(200, 200, 200))
                draw = ImageDraw.Draw(img)
                try:
                    font = ImageFont.truetype("Aptos.ttf", 24)
                except:
                    font = ImageFont.load_default()
                draw.text((150, 180), f"{subcategory}\n(Example Image)", fill=(0, 0, 0), font=font)
            
            self.original_image = img.copy()
            self.current_image = img
            self.image_path = img_path
            self.show_image(img)
        except Exception as e:
            messagebox.showerror("Error", f"Could not load image: {str(e)}")

    def show_image(self, image):
        """Display an image in the main window"""
        self.clear_window(keep_background=True)

        ttk.Button(self.root, text="← Back to Menu", command=self.create_main_menu).pack(anchor="nw", padx=10, pady=10)

        width, height = image.size
        max_size = (600, 400)
        
        if width > max_size[0] or height > max_size[1]:
            ratio = min(max_size[0]/width, max_size[1]/height)
            new_size = (int(width * ratio), int(height * ratio))
            image = image.resize(new_size, Image.LANCZOS)

        img_tk = ImageTk.PhotoImage(image)

        if self.image_label:
            self.image_label.destroy()
        self.image_label = ttk.Label(self.root, image=img_tk)
        self.image_label.image = img_tk
        self.image_label.pack(pady=10)

        tools_frame = ttk.LabelFrame(self.root, text="Enhancement Tools", padding=10)
        tools_frame.pack(fill="x", padx=10, pady=5)

        ttk.Button(tools_frame, text="Basic Transformations", 
                  command=self.show_basic_transformations).pack(side="left", padx=5)
        ttk.Button(tools_frame, text="Advanced Transformations", 
                  command=self.show_advanced_transformations).pack(side="left", padx=5)
        ttk.Button(tools_frame, text="Reset to Original", 
                  command=self.reset_image).pack(side="left", padx=5)

    def show_basic_transformations(self):
        """Show basic image transformations"""
        if not self.current_image:
            messagebox.showinfo("No Image", "Please select an artwork first")
            return
            
        img = cv2.cvtColor(np.array(self.current_image), cv2.COLOR_RGB2BGR)
        rows, cols, _ = img.shape

        M_translate = np.float32([[1, 0, 50], [0, 1, 100]])
        translated = cv2.warpAffine(img, M_translate, (cols, rows))

        center = (cols/2, rows/2)
        M_rotate = cv2.getRotationMatrix2D(center, 30, 0.8)
        rotated = cv2.warpAffine(img, M_rotate, (cols, rows))

        reflected = cv2.flip(img, 0)
        blurred = cv2.GaussianBlur(img, (7, 7), 0)

        transformations = {
            "Original": Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)),
            "Translated": Image.fromarray(cv2.cvtColor(translated, cv2.COLOR_BGR2RGB)),
            "Rotated": Image.fromarray(cv2.cvtColor(rotated, cv2.COLOR_BGR2RGB)),
            "Reflected": Image.fromarray(cv2.cvtColor(reflected, cv2.COLOR_BGR2RGB)),
            "Blurred": Image.fromarray(cv2.cvtColor(blurred, cv2.COLOR_BGR2RGB))
        }

        self.show_transformation_options(transformations)

    def show_advanced_transformations(self):
        """Show advanced image transformations"""
        if not self.current_image:
            messagebox.showinfo("No Image", "Please select an artwork first")
            return
            
        img = cv2.cvtColor(np.array(self.current_image), cv2.COLOR_RGB2BGR)
        rows, cols, _ = img.shape

        M_shear_x = np.float32([[1, 0.5, 0], [0, 1, 0], [0, 0, 1]])
        sheared_x = cv2.warpPerspective(img, M_shear_x, (int(cols*1.5), int(rows*1.5)))

        M_shear_y = np.float32([[1, 0, 0], [0.5, 1, 0], [0, 0, 1]])
        sheared_y = cv2.warpPerspective(img, M_shear_y, (int(cols*1.5), int(rows*1.5)))

        cropped = img[100:400, 100:500]
        median = cv2.medianBlur(img, 5)
        bilateral = cv2.bilateralFilter(img, 9, 75, 75)

        transformations = {
            "Sheared X": Image.fromarray(cv2.cvtColor(sheared_x, cv2.COLOR_BGR2RGB)),
            "Sheared Y": Image.fromarray(cv2.cvtColor(sheared_y, cv2.COLOR_BGR2RGB)),
            "Cropped": Image.fromarray(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)),
            "Median Blur": Image.fromarray(cv2.cvtColor(median, cv2.COLOR_BGR2RGB)),
            "Bilateral": Image.fromarray(cv2.cvtColor(bilateral, cv2.COLOR_BGR2RGB))
        }

        self.show_transformation_options(transformations)

    def show_transformation_options(self, transformations):
        """Display transformation options in a new window"""
        trans_window = tk.Toplevel(self.root)
        trans_window.title("Image Transformations")
        trans_window.geometry("800x600")

        notebook = ttk.Notebook(trans_window)
        notebook.pack(fill="both", expand=True)

        for name, img in transformations.items():
            frame = ttk.Frame(notebook)
            notebook.add(frame, text=name)
            
            img.thumbnail((500, 400))
            img_tk = ImageTk.PhotoImage(img)
            label = ttk.Label(frame, image=img_tk)
            label.image = img_tk
            label.pack(pady=10)
            
            ttk.Button(frame, text=f"Apply {name}", 
                      command=lambda i=img: self.apply_transformation(i)).pack(pady=5)
            ttk.Button(frame, text=f"Save {name}", 
                      command=lambda i=img, n=name: self.save_transformation(i, n)).pack(pady=5)

    def apply_transformation(self, transformed_img):
        """Apply the selected transformation to main view"""
        self.current_image = transformed_img
        self.show_image(transformed_img)
        messagebox.showinfo("Success", "Transformation applied to current view")

    def save_transformation(self, transformed_img, name):
        """Save the transformed image to file"""
        file_path = filedialog.asksaveasfilename(
            defaultextension=".jpg",
            filetypes=[("JPEG files", "*.jpg"), ("PNG files", "*.png"), ("All files", "*.*")],
            initialfile=f"{self.user_category}_{name}.jpg")

        if file_path:
            try:
                transformed_img.save(file_path)
                messagebox.showinfo("Success", f"Image saved as {file_path}")
            except Exception as e:
                messagebox.showerror("Error", f"Could not save image: {str(e)}")

    def reset_image(self):
        """Reset the image to its original state"""
        if self.original_image:
            self.current_image = self.original_image.copy()
            self.show_image(self.current_image)

    def clear_window(self, keep_background=False):
        """Clear all widgets from the window, optionally keeping the background"""
        for widget in self.root.winfo_children():
            if keep_background and widget == self.bg_label:
                continue
            widget.destroy()

    def logout(self):
        """Log out the current user"""
        self.current_user = None
        self.user_password = None
        self.user_category = None
        self.current_image = None
        self.original_image = None
        self.image_path = None
        self.create_login_screen()

def run_in_jupyter():
    print("YSMA Art Viewer, Now running...")
    root = tk.Tk()
    app = YSMAArtViewer(root)
    root.mainloop()

if __name__ == "__main__":
    if 'ipykernel' in sys.modules:
        run_in_jupyter()
    else:
        root = tk.Tk()
        app = YSMAArtViewer(root)
        root.mainloop()

YSMA Art Viewer, Now running...
